In [1]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

In [2]:
def load_array(data_arrays, batch_size, is_train=True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)
next(iter(data_iter))

[tensor([[ 0.5247,  0.9746],
         [ 0.7129,  0.1602],
         [-1.2279, -1.0457],
         [-0.5750, -0.7098],
         [ 0.9963, -0.2066],
         [ 1.0045,  0.2950],
         [-0.2935, -1.7651],
         [-0.4848,  1.5214],
         [-0.4113, -0.0208],
         [ 0.7269, -0.3101]]),
 tensor([[ 1.9414],
         [ 5.0831],
         [ 5.2776],
         [ 5.4478],
         [ 6.8901],
         [ 5.2174],
         [ 9.6276],
         [-1.9545],
         [ 3.4483],
         [ 6.7026]])]

In [3]:
from torch import nn

net = nn.Sequential(nn.Linear(2,1))

In [4]:
net[0].weight, net[0].bias

(Parameter containing:
 tensor([[0.4511, 0.4651]], requires_grad=True),
 Parameter containing:
 tensor([-0.4789], requires_grad=True))

In [5]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

In [6]:
loss = nn.MSELoss()

In [7]:
trainer = torch.optim.SGD(net.parameters(), lr = 0.03)

In [8]:
from tqdm import tqdm

num_epochs = 3
for epoch in range(num_epochs):
    pbar = tqdm(data_iter)
    for X, y in pbar:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
        pbar.set_description(f"Epoch {epoch}")
        pbar.set_postfix(loss=l, lr=0.03)
    with torch.no_grad():
        current_l = loss(net(features), labels)
        print(f'Epoch {epoch + 1}, loss: {current_l:.6f}')
    

Epoch 0: 100%|██████████| 100/100 [00:00<00:00, 427.41it/s, loss=tensor(0.0002, grad_fn=<MseLossBackward0>), lr=0.03]   


Epoch 1, loss: 0.000142


Epoch 1: 100%|██████████| 100/100 [00:00<00:00, 609.34it/s, loss=tensor(6.4622e-05, grad_fn=<MseLossBackward0>), lr=0.03]


Epoch 2, loss: 0.000094


Epoch 2: 100%|██████████| 100/100 [00:00<00:00, 598.05it/s, loss=tensor(9.2881e-05, grad_fn=<MseLossBackward0>), lr=0.03]


Epoch 3, loss: 0.000094


In [9]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([-0.0005, -0.0013])
b的估计误差： tensor([0.0001])
